In [ ]:
# --- CÉLULA 1: Setup e Carregamento de Bibliotecas ---

import pandas as pd

import numpy as np

import joblib

import os

# Bibliotecas de Pré-processamento e Métricas

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

from sklearn.svm import SVC


print("Bibliotecas importadas e prontas para Classificação Margin and Distance (Pima Indians Diabetes).")



Bibliotecas importadas e prontas para Classificação Ensemble (Pima Indians Diabetes).


In [4]:
# --- CÉLULA 2: Carregamento de Dados Pima (Arquivos Salvos no Notebook 03) ---

DATA_DIR = 'data'

# Carregamento dos conjuntos padronizados e separados 
try:
    X_train_scaled = joblib.load(os.path.join(DATA_DIR, 'X_train_pima_scaled.joblib')) # <-- Define X_train_scaled
    X_test_scaled = joblib.load(os.path.join(DATA_DIR, 'X_test_pima_scaled.joblib'))
    y_train = joblib.load(os.path.join(DATA_DIR, 'y_train_pima.joblib')) # <-- Define y_train
    y_test = joblib.load(os.path.join(DATA_DIR, 'y_test_pima.joblib'))
    print("Dados Pima Indians Diabetes (scaled) carregados com sucesso do diretório 'data/'.")
except Exception as e:
    # Se os arquivos não existirem, será necessário rodar a Célula 2 do Notebook 03
    print(f"ERRO: Não foi possível carregar os arquivos. Verifique se a Célula 2 do Notebook 03 foi executada. Erro: {e}")

print(f"Shape do X_train: {X_train_scaled.shape}")

Dados Pima Indians Diabetes (scaled) carregados com sucesso do diretório 'data/'.
Shape do X_train: (537, 8)


In [5]:
# --- CÉLULA 6: Support Vector Machine (SVM) Classifier ---
from sklearn.svm import SVC

# 1. Definir o range de hiperparâmetros
param_grid_svm = {
    'C': [0.1, 1.0, 10.0], # Parâmetro de regularização
    'kernel': ['linear', 'rbf'] # Tipos de kernel
}

# 2. Configurar o GridSearchCV
# Usamos probability=True para garantir que a função predict_proba funcione (necessário para AUC-ROC)
svm = SVC(probability=True, random_state=42)
svm_grid = GridSearchCV(
    estimator=svm,
    param_grid=param_grid_svm,
    scoring='roc_auc', # Otimizamos para AUC-ROC
    cv=5, 
    verbose=0,
    n_jobs=-1
)

# 3. Treinar
# O SVM é altamente sensível à escala, por isso usamos os dados SCALED
svm_grid.fit(X_train_scaled, y_train)

# 4. Salvar o melhor estimador
best_svm_model = svm_grid.best_estimator_
MODEL_DIR = 'models'
MODEL_FILENAME = 'svm_classifier_pima.joblib'
MODEL_PATH_SVM = os.path.join(MODEL_DIR, MODEL_FILENAME)
joblib.dump(best_svm_model, MODEL_PATH_SVM)
print(f"\nModelo SVM Classifier otimizado e salvo em: {MODEL_PATH_SVM}")

# 5. Avaliação
y_pred_proba_svm = best_svm_model.predict_proba(X_test_scaled)[:, 1] # Probabilidade da Classe 1
y_pred_svm = best_svm_model.predict(X_test_scaled) # Previsão da classe

auc_svm = roc_auc_score(y_test, y_pred_proba_svm)
accuracy_svm = accuracy_score(y_test, y_pred_svm)

print(f"\nSVM: Melhor Parâmetro encontrado: {svm_grid.best_params_}")
print(f"SVM: Acurácia (Teste): {accuracy_svm:.4f}")
print(f"SVM: AUC-ROC (Teste): {auc_svm:.4f}")
print("\nRelatório de Classificação (SVM):")
print(classification_report(y_test, y_pred_svm))


Modelo SVM Classifier otimizado e salvo em: models\svm_classifier_pima.joblib

SVM: Melhor Parâmetro encontrado: {'C': 10.0, 'kernel': 'linear'}
SVM: Acurácia (Teste): 0.7359
SVM: AUC-ROC (Teste): 0.8346

Relatório de Classificação (SVM):
              precision    recall  f1-score   support

           0       0.76      0.87      0.81       150
           1       0.67      0.49      0.57        81

    accuracy                           0.74       231
   macro avg       0.71      0.68      0.69       231
weighted avg       0.73      0.74      0.72       231

